In [ ]:
#from google.colab import files
import pandas as pd
import numpy as np
import calendar
# import warnings

import matplotlib.pyplot as plt

## Saldo en campo

In [ ]:
# Cargamos el archivo Excel - RFF Procesado
# --------------

ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_sabana_saldo_campo_pw = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_saldo_campo/182329be-0e9a-43ee-saldo-campo-pw.xlsx'
path_sabana_saldo_campo_sh = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_saldo_campo/19065b0d-5048-40f8-saldo-campo-sh.xlsx'

path_sabana_saldo_campo_pw_dias = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_saldo_campo/5d3cd28b-1322-479a-saldo-campo-pw-dias.xlsx'
path_sabana_saldo_campo_sh_dias = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_saldo_campo/5ecff3d5-57a8-4803-saldo-campo-sh-dias.xlsx'


# Cargar la hoja 2 (indexada desde 0, por lo tanto la hoja 2 es 'sheet_name=1')
df_sabana_saldo_campo_pw = pd.read_excel(ruta_path + path_sabana_saldo_campo_pw,sheet_name='Sheet1')
df_sabana_saldo_campo_sh = pd.read_excel(ruta_path + path_sabana_saldo_campo_sh,sheet_name='Sheet1')

df_sabana_saldo_campo_pw_dias = pd.read_excel(ruta_path + path_sabana_saldo_campo_pw_dias,sheet_name='Sheet1')
df_sabana_saldo_campo_sh_dias = pd.read_excel(ruta_path + path_sabana_saldo_campo_sh_dias,sheet_name='Sheet1')

# Eliminar las filas donde cualquiera de las columnas tenga el valor 'Totales' o NaN
df_sabana_saldo_campo_pw = df_sabana_saldo_campo_pw[~df_sabana_saldo_campo_pw[["ATCnombreMESR", "Date(ATCFechaR, 'DD/MM/YYYY')"]].isin(['Totales']).any(axis=1) & df_sabana_saldo_campo_pw[["ATCnombreMESR", "Date(ATCFechaR, 'DD/MM/YYYY')"]].notna().all(axis=1)]
df_sabana_saldo_campo_sh = df_sabana_saldo_campo_sh[~df_sabana_saldo_campo_sh[["ATCnombreMESR", "Date(ATCFechaR, 'DD/MM/YYYY')"]].isin(['Totales']).any(axis=1) & df_sabana_saldo_campo_sh[["ATCnombreMESR", "Date(ATCFechaR, 'DD/MM/YYYY')"]].notna().all(axis=1)]

df_sabana_saldo_campo_pw['PLANTACION'] = 'Palmawasi'
df_sabana_saldo_campo_sh['PLANTACION'] = 'Shanuzi'
df_sabana_saldo_campo_pw_dias['PLANTACION'] = 'Palmawasi'
df_sabana_saldo_campo_sh_dias['PLANTACION'] = 'Shanuzi'


# Concatenar los DataFrames
df_sabana_saldo_campo_full = pd.concat([df_sabana_saldo_campo_pw, df_sabana_saldo_campo_sh], ignore_index=True)
df_sabana_saldo_campo_dias_full = pd.concat([df_sabana_saldo_campo_pw_dias, df_sabana_saldo_campo_sh_dias], ignore_index=True)

df_sabana_saldo_campo_dias_full = df_sabana_saldo_campo_dias_full.rename(columns={'FECHA EVALUADA':'FECHA'})



# Creando variable FECHA
df_sabana_saldo_campo_full['FECHA'] = pd.to_datetime(df_sabana_saldo_campo_full["Date(ATCFechaR, 'DD/MM/YYYY')"])

print(df_sabana_saldo_campo_full.dtypes)

df_sabana_saldo_campo_full.head(10)



In [ ]:
df_sabana_saldo_campo_dias_full.head()

In [ ]:
# df_sabana_saldo_campo_full[['FECHA','PLANTACION','SALDO','SALDO > 3']].head()
df_sabana_saldo_campo_dias_full[['FECHA','PLANTACION','DIA 1','DIA 2','DIA 3','DIA 4','DIA 5','DIA 6','DIA 7']].head()

In [ ]:
# Unir los DataFrames en base a 'FECHA' y 'PLANTACION'
df_sabana_saldo_campo_full_vf = pd.merge(df_sabana_saldo_campo_full[['FECHA','PLANTACION','SALDO','SALDO > 3']],
                                         df_sabana_saldo_campo_dias_full[['FECHA','PLANTACION','>3 DIAS','DIA 1','DIA 2','DIA 3','DIA 4','DIA 5','DIA 6','DIA 7']],
                                         on=['FECHA', 'PLANTACION'], how='inner')
df_sabana_saldo_campo_full_vf.head(10)

### Guardamos la data de saldos

In [ ]:
# Guardamos la data en processed
# -----
# Especifica la ruta donde deseas guardar el CSV
ruta_sabana_saldo_campo_full_vf =  '/PROYECTO_001_analitica_descriptiva_TEA/data/processed/df_sabana_saldo_campo_full_vf.csv'  # Cambia esta ruta a la que necesites
# Guardar el DataFrame en formato CSV
df_sabana_saldo_campo_full_vf.to_csv(ruta_path + ruta_sabana_saldo_campo_full_vf, index=False)
print(f'Datos guardados en {ruta_path + ruta_sabana_saldo_campo_full_vf}')

df_sabana_saldo_campo_full_vf.head(10)

In [ ]:
# Filtramos la tendencia de interes
# -------------

# ---
df_sabana_saldo_campo_full_tendencia = df_sabana_saldo_campo_full[df_sabana_saldo_campo_full['PLANTACION'] == 'Palmawasi']
df_sabana_saldo_campo_full_tendencia = df_sabana_saldo_campo_full_tendencia.sort_values(by='FECHA')
# ---

plantacion_name = df_sabana_saldo_campo_full['PLANTACION'].unique()

# Crear el gráfico de líneas para 'toneladas_dia'
plt.figure(figsize=(15, 5))
plt.plot(df_sabana_saldo_campo_full_tendencia['FECHA'], df_sabana_saldo_campo_full_tendencia['SALDO'], marker='o', markersize=2, linestyle='-', label='SALDO')
plt.plot(df_sabana_saldo_campo_full_tendencia['FECHA'], df_sabana_saldo_campo_full_tendencia['SALDO > 3'], marker='o', markersize=2, linestyle='-', label='SALDO > 3')
plt.title('RFF Procesado por Día a través del Tiempo - '+ plantacion_name[0])
plt.xlabel('Fecha')
plt.ylabel('RFF Procesado por Día')
plt.grid(True)
plt.legend()
#plt.xticks(rotation=45)

# Mostrar la gráfica
print(df_sabana_saldo_campo_full_tendencia.head())
plt.tight_layout()
plt.show()